In [1]:
%run 0.0_Dependance.ipynb

In [2]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from sklearn.preprocessing import RobustScaler

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = RobustScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy='not majority',random_state=42)

X_train, y_train = smote.fit_resample(X_train, y_train)

In [3]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
import numpy as np
from joblib import Parallel, delayed
from joblib import dump,load

def load_create_dftop(df_path, n_top):
    df = load(df_path)
    df_sorted = df.sort_values(by="LogLoss_mean", ascending=False)
    df_top = df_sorted.iloc[:n_top, :]
    return df_top

# Fonction pour entraîner le modèle et calculer le log_loss
def compute_log_loss(i, df, X_train, y_train, X_test, y_test):
    dico = df.iloc[i, :-2].to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    return log_loss(y_test, probs)

df_score_random_v6_top_1000 = load_create_dftop("df_score_random_v1_1.joblib", 1000)

# Parallélisation du calcul du log_loss
log_losses = Parallel(n_jobs=-1)(delayed(compute_log_loss)(i, df_score_random_v6_top_1000, X_train, y_train, X_test, y_test) for i in range(1000))

df_score_random_v6_top_1000['Test_log_loss'] = log_losses
df_score_random_v6_top_1000 = df_score_random_v6_top_1000.sort_values(by="Test_log_loss",ascending=True)
dump(df_score_random_v6_top_1000,"df_score_random_v6_top_1000.joblib")

['df_score_random_v6_top_1000.joblib']

In [4]:
df_score_random_v6_top_1000 = load("df_score_random_v6_top_1000.joblib")
df_score_random_v6_top_1000

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std,Test_log_loss
1336,400,6,0.06,0.7,0.75,0.2,7,0.5,0,multi:softprob,mlogloss,gbtree,-0.428443,0.012964,0.450749
13725,900,6,0.06,0.7,1,0.2,7,0.4,0,multi:softprob,mlogloss,gbtree,-0.428959,0.011701,0.450787
20894,400,6,0.07,0.7,0.75,0.2,8,0.1,2,multi:softprob,mlogloss,gbtree,-0.428499,0.012632,0.450877
18235,400,6,0.06,0.7,0.75,0.2,7,0.1,0,multi:softprob,mlogloss,gbtree,-0.427482,0.012619,0.450957
1065,400,6,0.06,0.5,1,0.2,10,0.5,1,multi:softprob,mlogloss,gbtree,-0.428791,0.013852,0.451024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12503,400,4,0.1,0.5,0.75,0.2,6,0.4,0,multi:softprob,mlogloss,gbtree,-0.429954,0.014308,0.463449
30062,800,5,0.1,0.7,0.75,0.2,5,0.5,1,multi:softprob,mlogloss,gbtree,-0.430446,0.014559,0.463492
16007,500,4,0.1,1,0.75,0.2,5,0.2,0,multi:softprob,mlogloss,gbtree,-0.430076,0.014084,0.463511
11401,600,5,0.1,0.7,0.75,0.2,6,0.5,0,multi:softprob,mlogloss,gbtree,-0.430242,0.014490,0.463578
